## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-27-14-56-16 +0000,bbc,Pubs given support package after business rate...,https://www.bbc.com/news/articles/c78vqj99168o...
1,2026-01-27-14-56-09 +0000,nypost,Southern California city on edge over viral vi...,https://nypost.com/2026/01/27/us-news/socal-ci...
2,2026-01-27-14-55-58 +0000,nyt,"Fed, Signaling Little Urgency, Prepares to Pau...",https://www.nytimes.com/2026/01/27/business/ec...
3,2026-01-27-14-55-09 +0000,nypost,Washington Post union urges staffers to appeal...,https://nypost.com/2026/01/27/media/washington...
4,2026-01-27-14-50-05 +0000,nypost,Ilhan Omar to huddle with constituents in Minn...,https://nypost.com/2026/01/27/us-news/ilhan-om...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2430/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
73,trump,65
63,ice,28
46,minneapolis,27
76,minnesota,25
137,pretti,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
17,2026-01-27-14-28-37 +0000,wapo,"Alex Pretti’s sister speaks out, as Trump send...",https://www.washingtonpost.com/nation/2026/01/...,154
188,2026-01-27-01-29-20 +0000,nypost,ICE tensions could be thawing in Minn. after T...,https://nypost.com/2026/01/26/us-news/ice-tens...,152
25,2026-01-27-14-15-00 +0000,wsj,President Trump’s pivot in Minneapolis came af...,https://www.wsj.com/politics/policy/the-48-hou...,143
72,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...,143
174,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...,141


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
17,154,2026-01-27-14-28-37 +0000,wapo,"Alex Pretti’s sister speaks out, as Trump send...",https://www.washingtonpost.com/nation/2026/01/...
132,89,2026-01-27-08-56-53 +0000,nypost,Anti-ICE protesters arrested outside Minnesota...,https://nypost.com/2026/01/27/us-news/anti-ice...
200,55,2026-01-27-00-42-00 +0000,wsj,The Army will gain access to Salesforce’s tech...,https://www.wsj.com/politics/national-security...
72,53,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...
174,51,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...
166,49,2026-01-27-03-46-00 +0000,wsj,Democrats want changes to the Department of Ho...,https://www.wsj.com/politics/policy/government...
73,48,2026-01-27-12-01-26 +0000,nyt,Trump Vows Higher Tariffs for South Korea Mont...,https://www.nytimes.com/2026/01/27/world/asia/...
289,43,2026-01-26-19-56-06 +0000,wapo,Six presumed dead in private jet crash at Main...,https://www.washingtonpost.com/transportation/...
30,41,2026-01-27-14-05-17 +0000,bbc,'Crying horse' toys go viral in China ahead of...,https://www.bbc.com/news/articles/c70l7zndy3jo...
312,35,2026-01-26-18-26-00 +0000,wsj,Israel Says It Will Reopen Gaza Border Crossin...,https://www.wsj.com/world/middle-east/israel-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
